In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from helpers import llm
from langchain.tools import tool

# OpenAI API Wrapper erstellen
llm = llm()

# Custom Function als Tool definieren


@tool
def test_tool(question: str):
    "This tool provides information about Fritz Karuugaa. Give a question as input."
    return "Fritz Karuugaa is a software developer"


# Tools müssen immer als list übergeben werden
tools = [test_tool]

# Tools an das llm binden. Dadurch werden automatisch die vollständige Tool beschreibung in das Prompting des LLM übergeben. (Funktioniert nur mit den großen LLM's OpenAI, Claude, etc.)
llm_with_tools = llm.bind_tools(tools)

# Prompt Template erstellen um dem Agenten einen System prompt mitzugeben.
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "Du bist ein hilfsbereiter Assistent. Löse deine Aufgaben auf eine humorvolle Art und weiße. Antworte zusätzlich immer mit einem Witz.",
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder("intermediate_steps"),
    ]
)

# Einfache LCEL Chain die das Prompt Template erstellt und in den LLM call injectet.
chain = prompt | llm_with_tools

In [ ]:
import operator
from typing import Annotated, TypedDict

from langchain_core.agents import AgentActionMessageLog, AgentFinish
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph

# State für den Graph definieren


class AgentState(TypedDict):
    input: str
    intermediate_steps: Annotated[list[tuple[AgentActionMessageLog, str]], operator.add]
    answer: str

# Continue Condition die entscheidet prüft ob ein Tool ausgeben werden soll oder der Graph beendet wird.


def should_continue(state):
    return "continue" if state["intermediate_steps"][-1].tool_calls else "end"

# Function um das LLM zu triggern und Agentenlogik auszuführen.


def call_model(state, config):
    response = chain.invoke(state, config=config)
    print("state", state)
    if isinstance(response, AgentFinish):
        return {"answer": response.answer}
    else:
        return {"intermediate_steps": [response]}
    # return {"intermediate_steps": [chain.invoke(state, config=config)]}

# Function um lokal die Tools auszuführen.


def _invoke_tool(tool_call):
    tool = {tool.name: tool for tool in tools}[tool_call["name"]]
    return ToolMessage(tool.invoke(tool_call["args"]), tool_call_id=tool_call["id"])


tool_executor = RunnableLambda(_invoke_tool)

# Function um die Tool Calls auszulesen und parallel auszuführen.


def call_tools(state):
    last_message = state["intermediate_steps"][-1]
    return {"intermediate_steps": tool_executor.batch(last_message.tool_calls)}


# Gestaltung des Grapen Workflows
workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")

# Graph kompilieren.
graph = workflow.compile()

# Graph mit input ausführen.
graph.invoke({"input": "Was weißt du über Fritz Karuugaa?"})